In [2]:
import numpy as np
import networkx as nx
from itertools import product
import random
import matplotlib.pyplot as plt

In [109]:
partition_table5a = [
    ["UUU", "UUC", "UUA"],
    ["UUG", "CUG", "AUG"],
    ["CUU", "CUC", "CUA"],
    ["AUU", "AUC", "AUA"],
    ["GUU", "GUC", "GUA"],
    ["GUG", "GCG", "GAG", "GGG"],
    ["UCU", "UCC", "UCA"],
    ["UCG", "CCG", "ACG"],
    ["CCU", "CCC", "CCA"],
    ["ACU", "ACC", "ACA"],
    ["GCU", "GCC", "GCA"],
    ["UAU", "UAC", "UAA"],
    ["UAG", "CAG", "AAG"],
    ["CAU", "CAC", "CAA"],
    ["AAU", "AAC", "AAA"],
    ["GAU", "GAC", "GAA"],
    ["UGU", "UGC", "UGA"],
    ["UGG", "CGG", "AGG"],
    ["CGU", "CGC", "CGA"],
    ["AGU", "AGC", "AGA"],
    ["GGU", "GGC", "GGA"],
]

In [110]:
print(len(set([item for sublist in partition_table5a for item in sublist])))

64


In [111]:
length = 3
alphabet = ['A', 'C', 'G', 'U']
nodes = [''.join(p) for p in product(alphabet, repeat=length)]

In [112]:
swap_index =  {
    0: {#U
        0: 
        #C    #A    #G   
        {1: 0, 2: 1, 3: 2},
        #C
        1:
        #A    #G
        {2: 3, 3: 4},
        #A
        2:
        #G
        {3: 5}
        },
    1: {#A
        0: 
        #C    #A    #G   
        {1: 6, 2: 7, 3: 8},
        #C
        1:
        #A    #G
        {2: 9, 3: 10},
        #A
        2:
        #G
        {3: 11}
        },
    2: {#A
        0: 
        #C    #A    #G   
        {1: 12, 2: 13, 3: 14},
        #C
        1:
        #A    #G
        {2: 15, 3: 16},
        #A
        2:
        #G
        {3: 17}
        },
}

table_ones = np.ones(18)

table_altered = [1, 1, 1, 1, 1, 1,
                 1, 1, 1, 1, 1, 1,
                 2, 4, 2, 2, 4, 2]

In [24]:
def weights(s1, s2, table):
    if sum(a != b for a, b in zip(s1, s2)) == 1:
        if s1[0] != s2[0]:
            # base 1
            letter1 = s1[0]
            letter2 = s2[0]
            base = 0
        elif s1[1] != s2[1]:
            # base 2
            letter1 = s1[1]
            letter2 = s2[1]
            base = 1
        else:
            # base3
            letter1 = s1[2]
            letter2 = s2[2]
            base = 2
    
        l1 = ['U', 'C', 'A', 'G'].index(letter1)
        l2 = ['U', 'C', 'A', 'G'].index(letter2)
        if l1 > l2:
            l1, l2 = l2, l1
        return table[swap_index[base][l1][l2]]

In [25]:
def conductance(S, G):
    numerator = 0
    denominator = 0
    for edge, w in nx.get_edge_attributes(G, "weight").items():
        node1, node2 = edge
        if node1 in S and node2 in S:
            denominator += 2 * w
        else:
            numerator += w
            denominator += w

    return numerator / denominator

In [117]:
def mean_conductance(table, code):
    conductances = []
    for S in code:
        G = nx.Graph()
        for i, node1 in enumerate(S):
            for node2 in nodes:
                if node2 == node1:
                    continue
                w = weights(node1, node2, table)
                if w:
                    G.add_edge(node1, node2, weight=w)
                    
        conductances.append(conductance(S, G))
    return np.mean(conductances)

In [122]:
def fitness(table, code):
    return 1 - mean_conductance(table, code)

In [123]:
print(mean_conductance(table_ones, partition_table5a))
print(fitness(table_ones, partition_table5a))

0.7724867724867727
0.22751322751322733


In [158]:
def random_partition():
    partition = []
    for i in range(21):
        partition.append(i)
    for i in range(64 - 21):
        partition.append(random.randint(0, 20))
    random.shuffle(partition)
    return partition

def partition_to_codons(partition):
    partition_codons = [[] for _ in range(21)]
    for i, group in enumerate(partition):
        partition_codons[group].append(nodes[i])
    return partition_codons

In [163]:
partition = partition_to_codons(random_partition())
print(mean_conductance(table_ones, partition))

0.973721340388007


In [200]:
def initialize_population(pop_size=100):
    partitions = [random_partition() for _ in range(pop_size)]
    return partitions

In [251]:
def crossover(parent1, parent2):
    child = parent1.copy()
    for i in range(64):
        if np.random.rand() > 0.5:
            child[i] = parent2[i]
        if len(set(child)) != 21:
            child[i] = parent1[i]
    return child

p1 = random_partition()
p2 = random_partition()

print(crossover(p1, p2))

[17, 12, 11, 13, 3, 19, 1, 12, 13, 5, 14, 15, 5, 11, 16, 3, 9, 19, 18, 19, 16, 0, 12, 9, 14, 18, 7, 16, 5, 12, 3, 3, 7, 2, 20, 17, 6, 2, 11, 0, 12, 18, 12, 3, 0, 8, 18, 4, 10, 4, 11, 11, 20, 13, 17, 10, 13, 16, 7, 2, 19, 0, 8, 17]


In [252]:
def mutate(parent, mutation_rate=0.4):
    child = parent.copy()
    if np.random.rand() < mutation_rate:
        return child
    
    while True:
        i, j = np.random.choice(64, 2, replace=False)
        child[i], child[j] = parent[j], parent[i]
        if len(set(child)) == 21:
            break
        else:
            child[i], child[j] = parent[i], parent[j]
    return child
        
p = random_partition()

In [266]:
def evolutionary_algorithm(fitness, pop_size=150, crossover_rate=0.6, max_generations=1000, bias_factor=1, table=table_ones, start=None):
    
    if start is not None:
        population = [start for _ in range(pop_size)]
    else:
        population = initialize_population(pop_size)
    
    best_fitness = 0
    generations = 0
    
    probabilities = np.exp(-bias_factor * np.arange(pop_size) / pop_size)
    probabilities /= np.sum(probabilities)
    
    while generations < max_generations:
        fitness_values = np.array([fitness(table, partition_to_codons(ind)) for ind in population])
        sorted_indices = np.argsort(fitness_values)[::-1]
        sorted_population = []
        for i in sorted_indices:
            sorted_population.append(population[i])
        population = sorted_population
        new_population = []
      
        for i in range(int(crossover_rate * pop_size)):
            parent1_index, parent2_index = np.random.choice(pop_size, size=2, p=probabilities, replace=False)
            parent1, parent2 = population[parent1_index], population[parent2_index]
            child = crossover(parent1, parent2)
            new_population.append(child)
        
        while len(new_population) < pop_size:
            new_population.append(population[np.random.choice(pop_size, p=probabilities)].copy())
        
        new_population = np.array([mutate(ind) for ind in new_population])
        
        new_best_fitness = max(fitness_values)
        if new_best_fitness > best_fitness:
            best_fitness = new_best_fitness
        print(f"{best_fitness:.5f}, {new_best_fitness:.5f}", generations, f"cond: {(1 - best_fitness):.5f}")
        
        population = new_population
        generations += 1
    
    fitness_values = np.array([fitness(table, partition_to_codons(ind)) for ind in population])
    sorted_indices = np.argsort(fitness_values)[::-1]
    population = population[sorted_indices]
    best_individual = population[0]
    
    return best_individual

In [267]:
best_ind = evolutionary_algorithm(fitness, pop_size=100, max_generations=100, bias_factor=2.0, table=table_altered)

0.07266, 0.07266 0 cond: 0.92734
0.07460, 0.07460 1 cond: 0.92540
0.07460, 0.07234 2 cond: 0.92540
0.09456, 0.09456 3 cond: 0.90544
0.09456, 0.08492 4 cond: 0.90544
0.09456, 0.08265 5 cond: 0.90544
0.09456, 0.08265 6 cond: 0.90544
0.09694, 0.09694 7 cond: 0.90306
0.09694, 0.07472 8 cond: 0.90306
0.09694, 0.07812 9 cond: 0.90306
0.09694, 0.08993 10 cond: 0.90306
0.09694, 0.09025 11 cond: 0.90306
0.09694, 0.08447 12 cond: 0.90306
0.09694, 0.08923 13 cond: 0.90306
0.09694, 0.08673 14 cond: 0.90306
0.10680, 0.10680 15 cond: 0.89320
0.10680, 0.10493 16 cond: 0.89320
0.10680, 0.10096 17 cond: 0.89320
0.10680, 0.10578 18 cond: 0.89320
0.10680, 0.10578 19 cond: 0.89320
0.10850, 0.10850 20 cond: 0.89150
0.10850, 0.10850 21 cond: 0.89150
0.10850, 0.10850 22 cond: 0.89150
0.13108, 0.13108 23 cond: 0.86892
0.13108, 0.10873 24 cond: 0.86892
0.13175, 0.13175 25 cond: 0.86825
0.13175, 0.11485 26 cond: 0.86825
0.13175, 0.12007 27 cond: 0.86825
0.13175, 0.12007 28 cond: 0.86825
0.13866, 0.13866 29 cond

In [269]:
best_ind = evolutionary_algorithm(fitness, pop_size=300, max_generations=50, bias_factor=5.0, table=table_altered, start=best_ind)

0.41134, 0.41134 0 cond: 0.58866
0.41134, 0.41134 1 cond: 0.58866
0.41134, 0.41134 2 cond: 0.58866
0.41134, 0.41134 3 cond: 0.58866
0.41134, 0.41134 4 cond: 0.58866
0.41134, 0.41134 5 cond: 0.58866
0.41134, 0.41134 6 cond: 0.58866
0.41134, 0.41134 7 cond: 0.58866
0.41134, 0.41134 8 cond: 0.58866
0.41134, 0.41134 9 cond: 0.58866
0.41134, 0.41134 10 cond: 0.58866
0.41134, 0.41134 11 cond: 0.58866
0.41134, 0.41134 12 cond: 0.58866
0.41134, 0.41134 13 cond: 0.58866
0.41134, 0.41134 14 cond: 0.58866
0.41134, 0.41134 15 cond: 0.58866
0.41134, 0.41134 16 cond: 0.58866
0.41134, 0.41134 17 cond: 0.58866
0.41134, 0.41134 18 cond: 0.58866
0.41134, 0.41134 19 cond: 0.58866
0.41134, 0.41134 20 cond: 0.58866
0.42177, 0.42177 21 cond: 0.57823
0.42177, 0.42177 22 cond: 0.57823
0.42177, 0.42177 23 cond: 0.57823
0.42177, 0.42177 24 cond: 0.57823
0.42177, 0.42177 25 cond: 0.57823
0.42177, 0.42177 26 cond: 0.57823
0.42177, 0.42177 27 cond: 0.57823
0.42177, 0.42177 28 cond: 0.57823
0.42177, 0.42177 29 cond

In [271]:
print(partition_to_codons(best_ind))

[['CUA', 'CUC', 'CUG', 'CUU'], ['UAA', 'UAC', 'UAG', 'UAU'], ['CCA', 'CCC', 'CCG', 'CCU'], ['CAA', 'CAC', 'CAG', 'CAU'], ['AUA', 'AUU'], ['CGA', 'CGU'], ['AUC', 'AUG', 'UGA', 'UGU'], ['UGC', 'UGG'], ['GUC', 'GUG'], ['AGA', 'AGC', 'AGG', 'AGU'], ['UUA', 'UUC', 'UUG', 'UUU'], ['GCA', 'GCU'], ['GAA', 'GAC', 'GAG', 'GAU'], ['ACA', 'ACC', 'ACG', 'ACU'], ['CGC', 'CGG'], ['GGA', 'GGC', 'GGG', 'GGU'], ['GCC', 'GCG'], ['AAA', 'AAU'], ['AAC', 'AAG'], ['UCA', 'UCC', 'UCG', 'UCU'], ['GUA', 'GUU']]


In [ ]:
print(partition_to_codons(best_ind))